In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-08-12 15:08:04 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-08-12 15:08:04 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-08-12 15:08:05 __main__ INFO     transformers.__version__='4.54.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-08-12 15:08:07 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-12 15:08:07 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-12 15:08:07 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-08-12 15:08:07 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-08-12 15:08:08 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-08-12 15:08:08 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-08-12 15:08:08 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-08-12 15:08:08 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-08-12 15:08:08 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-08-12 15:08:55 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-08-12 15:08:55 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-08-12 15:08:55 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


## Selection Task

In [5]:
from src.selection.data import SelectOneTask

select_prof = SelectOneTask.load(
    path="/disk/u/arnab/Codes/Projects/retrieval/data_save/selection/profession.json"
)

print(select_prof)

SelectOneTask: (profession of a famous person)
Categories: actor(20), singer(20), comedian(20), director(20), basketball player(20), football player(20), soccer player(20), tennis player(20), golfer(20), boxer(20), news anchor(20), journalist(20), author(20), fashion designer(20), entrepreneur(19), politician(20)



In [6]:
from typing import Literal

output_formatting: Literal[
    "zero_shot", # no formatting, model preference
    "object", # Bill Gates
    "lettered", # a. Bill Gates
] = "zero_shot"

In [7]:
prompt_template_idx = 1
option_style = "numbered"  # "numbered", "lettered", "ordinal"

# one_shot = select_prof.get_random_sample(
#     mt = mt,
#     prompt_template_idx=prompt_template_idx,
#     option_style="numbered",
#     category="actor",
#     filter_by_lm_prediction = False,
# )

# print(one_shot)

sample = select_prof.get_random_sample(
    mt = mt,
    obj_idx=3,
    prompt_template_idx=prompt_template_idx,
    option_style=option_style,
    category="actor",
    filter_by_lm_prediction = True,
    # output_formatting=output_formatting,
)
print(sample)

Angelina Jolie -> Al Pacino (3): ['Dan Brown', 'Terence Crawford', 'Chris Rock', 'Al Pacino', 'Doja Cat', 'Michael Kors']


In [8]:
# sample.prompt_template = select_prof.prompt_templates[1]

print(f'"{sample.prompt()}"', ">>", sample.obj)

"a. Dan Brown
b. Terence Crawford
c. Chris Rock
d. Al Pacino
e. Doja Cat
f. Michael Kors
Who among these people mentioned above share the same profession as Angelina Jolie?
Answer:" >> Al Pacino


In [9]:
# print(sample.prompt(option_style="single_line"))

In [10]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.obj)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


" d. Al Pacino
Explanation: Angelina Jolie is an actress, and Al Pacino" >> Al Pacino


In [11]:
from src.functional import prepare_input, get_hs, interpret_logits
from src.selection.utils import get_first_token_id, verify_correct_option

# inputs = prepare_input(prompts=sample.prompt(), tokenizer=mt)
# logit_module = (mt.lm_head_name, -1)
# logits = get_hs(
#     mt=mt,
#     input=inputs,
#     locations=[logit_module],
#     return_dict=False,
# ).squeeze()

verify_correct_option(
    mt=mt,
    # logits=logits,
    target=sample.obj,
    options=sample.options,
    input=sample.prompt()
)

(True,
 [PredictedToken(token=' d', prob=0.416015625, logit=19.5, token_id=294, metadata=None),
  PredictedToken(token=' (', prob=0.173828125, logit=18.625, token_id=320, metadata=None),
  PredictedToken(token=' a', prob=0.11962890625, logit=18.25, token_id=264, metadata=None),
  PredictedToken(token=' Only', prob=0.072265625, logit=17.75, token_id=8442, metadata=None),
  PredictedToken(token=' Al', prob=0.0341796875, logit=17.0, token_id=1708, metadata=None)],
 OrderedDict([(1708,
               (5,
                PredictedToken(token=' Al', prob=0.0341796875, logit=17.0, token_id=1708, metadata=None))),
              (11824,
               (9,
                PredictedToken(token=' Dan', prob=0.00921630859375, logit=15.6875, token_id=11824, metadata=None))),
              (8096,
               (24,
                PredictedToken(token=' Michael', prob=0.00150299072265625, logit=13.875, token_id=8096, metadata=None))),
              (3234,
               (26,
                Predicte

In [12]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.subj,
    mt = mt,
    heads = HEADS
)

0 patches to ablate possible answer information from options
2025-08-12 15:09:05 src.experiments.utils DEBUG    Predictions: ['" d"[294] (p=0.416, logit=19.500)', '" ("[320] (p=0.174, logit=18.625)', '" a"[264] (p=0.120, logit=18.250)', '" Only"[8442] (p=0.072, logit=17.750)', '" Al"[1708] (p=0.034, logit=17.000)']
2025-08-12 15:09:05 src.experiments.utils INFO     Combined attention matrix for all heads


## Odd one out

In [30]:
from src.selection.data import SelectOddOneOutTask

odd_one_out = SelectOddOneOutTask.load(
    path=os.path.join(env_utils.DEFAULT_DATA_DIR, "selection/profession.json")
)

print(odd_one_out)

SelectOddOneOutTask: (profession of a famous person)
Categories: actor(20), singer(20), comedian(20), director(20), basketball player(20), football player(20), soccer player(20), tennis player(20), golfer(20), boxer(20), news anchor(20), journalist(20), author(20), fashion designer(20), entrepreneur(19), politician(20)



In [31]:
sample = odd_one_out.get_random_sample(
    mt = mt,
    obj_idx=3,
    prompt_template_idx=3,
    option_style=option_style,
    # category="actor",
    filter_by_lm_prediction = False,
    # output_formatting=output_formatting,
)
print(sample)
print(f'"{sample.prompt()}"', ">>", sample.obj)

Carl Bernstein -> Manny Pacquiao (3): ['Yamiche Alcindor', 'Glenn Greenwald', 'Wesley Lowery', 'Manny Pacquiao', 'David Remnick', 'Jane Mayer']
"a. Yamiche Alcindor
b. Glenn Greenwald
c. Wesley Lowery
d. Manny Pacquiao
e. David Remnick
f. Jane Mayer
Who among these people mentioned above is NOT a journalist by profession?
Answer:" >> Manny Pacquiao


In [32]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.obj)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


" d. Manny Pacquiao
Manny Pacquiao is a professional boxer and a politician." >> Manny Pacquiao


In [33]:
verify_correct_option(
    mt=mt,
    # logits=logits,
    target=sample.obj,
    options=sample.options,
    input=sample.prompt()
)

(True,
 [PredictedToken(token=' d', prob=0.5546875, logit=21.5, token_id=294, metadata=None),
  PredictedToken(token=' Manny', prob=0.296875, logit=20.875, token_id=95255, metadata=None),
  PredictedToken(token=' (', prob=0.12353515625, logit=20.0, token_id=320, metadata=None),
  PredictedToken(token=' The', prob=0.00787353515625, logit=17.25, token_id=578, metadata=None),
  PredictedToken(token=' D', prob=0.00543212890625, logit=16.875, token_id=423, metadata=None)],
 OrderedDict([(95255,
               (2,
                PredictedToken(token=' Manny', prob=0.296875, logit=20.875, token_id=95255, metadata=None))),
              (6941,
               (28,
                PredictedToken(token=' David', prob=0.00010585784912109375, logit=12.9375, token_id=6941, metadata=None))),
              (816,
               (29,
                PredictedToken(token=' Y', prob=9.918212890625e-05, logit=12.875, token_id=816, metadata=None))),
              (58706,
               (40,
               

In [34]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.subj,
    mt = mt,
    heads = HEADS
)

0 patches to ablate possible answer information from options
2025-08-12 15:16:29 src.experiments.utils DEBUG    Predictions: ['" d"[294] (p=0.555, logit=21.500)', '" Manny"[95255] (p=0.297, logit=20.875)', '" ("[320] (p=0.124, logit=20.000)', '" The"[578] (p=0.008, logit=17.250)', '" D"[423] (p=0.005, logit=16.875)']
2025-08-12 15:16:29 src.experiments.utils INFO     Combined attention matrix for all heads


## Counting Task

In [ ]:
from src.selection.data import CountingTask

counting_fruits = CountingTask.load(
    path="../data_save/counting/fruits.json"
)

print(counting_fruits)

In [ ]:
sample = counting_fruits.get_random_sample(
    mt = mt,
    prompt_template_idx=0,
    option_style="single_line",
    category="fruits",
    filter_by_lm_prediction=True,
    n_count=2,
    n_distractors=3
)

print(sample)

In [ ]:
sample.prompt_template = counting_fruits.prompt_templates[1]

print(f'"{sample.prompt()}"', ">>", sample.count)

In [ ]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.count)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.category,
    mt = mt,
    heads = HEADS
)

## Deduction Task

In [ ]:
from src.selection.data import DeductionTask

deduction_task = DeductionTask.load(
    dir_path="../data_save/deduction"
)

print(deduction_task)

In [ ]:
sample = deduction_task.get_random_sample(
    mt = mt,
    topic_name = "height",
    depth = 5,
)

print(sample)

In [ ]:
from src.functional import generate_with_patch

generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt,
    options = ["Alice", "Bob", "Cam", "Dave", "Eli"],
    pivot = sample.answer,
    mt = mt,
    heads = HEADS
)

## All of the Above Selection

In [ ]:
from src.selection.data import SelectAllTask

select_all_prof = SelectAllTask.load(
    path="../data_save/selection/profession.json"
)

print(select_all_prof)

In [ ]:
sample = select_all_prof.get_random_sample(
    mt=mt
)

print(sample)

In [ ]:
from src.functional import generate_with_patch

generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    n_gen_per_prompt=1,
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.category,
    mt = mt,
    heads = HEADS
)

## MISC

In [ ]:
from itertools import product
import random

all_heads = list(product(range(mt.n_layer), range(mt.config.num_attention_heads)))

random_heads = random.sample(
    all_heads,
    k = 50
)
random_heads

In [ ]:
# instruction = """Instructions: 1. Track the belief of each character as described in the story. 2. A character's belief is formed only when they perform an action themselves or can observe the action taking place. 3. A character does not have any belief about the container or its content which they cannot observe directly. 4. To answer the question, predict only the final state of the queried container in fewest tokens possible, strictly based on the belief of the character, mentioned in the question. 5. Do not predict the entire sentence with character or container as the final output.\n\n"""

# prompt = instruction + """Story: Karen and Max are working in a busy restaurant. To complete an order, Karen grabs an opaque flute and fills it with soda. Then Max grabs another opaque jar and fills it with coffee. Max cannot observe Karen's actions. Karen cannot observe Max's actions.
# Question: What does Karen believe the jar contains?
# Answer:"""

# prompt = """1. Bus
# 2. Book
# 3. Cup
# 4. Plate
# 5. Glass
# 6. None of the above
# Which one of the objects mentioned above is a fruit?
# Answer:"""

# prompt = """1. Peach
# 2. Apple
# 3. Banana
# 4. Orange
# 5. Grapes
# 6. All of the above
# Which one of the objects mentioned above is a fruit?
# Answer:"""

prompt = """Items: Tea, Mango, Coffee, Orange, Transistor, Water, Kiwi, Cup
Find the sixth item in the list?
Answer:"""

verify_head_patterns(
    prompt=prompt,
    options=[
        "Tea",
        "Mango",
        "Coffee",
        "Orange",
        "Juice",
        "Water",
        "Kiwi",
        "Cup"
    ],
    # ablate_possible_ans_info_from_options=True,
    pivot="cup",
    mt=mt,
    heads=HEADS,
    # heads = [(33, 18), (35, 19), (47, 18)],
    # visualize_individual_heads=True,
    # heads= random_heads,
    # generate_full_answer=True,
    bare_prompt_template=" {}"
)